In [ ]:
from typing import List
from numpy import ndarray

import cv2

import numpy as np

from aipose.plot import plot
from aipose.models.yolov7 import YoloV7PoseKeypoints
from aipose.stream import process_video
from aipose.frame import FrameYoloV7

In [ ]:
class CustomYoloV7PoseKeypointsFrameProcessor(FrameYoloV7):
    def __init__(self, path:str):
        super().__init__()
        self.resolution = (640,480)
        fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
        self.out = cv2.VideoWriter(path, fourcc, 25, self.resolution)

    def on_predict(
        self, frame: ndarray, prediction: List[YoloV7PoseKeypoints], image_tensor: ndarray
    ) -> None | ndarray:
        frame = self._plot(prediction, image_tensor)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, dsize=self.resolution, interpolation=cv2.INTER_CUBIC)
        self.out.write(frame)
        return frame

    def _plot(self, prediction: List[YoloV7PoseKeypoints], image_tensor: ndarray) -> ndarray:
        frame = plot(
            image_tensor,
            np.array([value.get_raw_keypoint() for value in prediction]),
            plot_image=False,
            return_img=True,
        )
        return frame

    def on_ends_stream(self) -> None:
        self.out.release()


custom_keypoints_frame_processor = CustomYoloV7PoseKeypointsFrameProcessor('./../video_processed.mp4')

In [ ]:
process_video(custom_keypoints_frame_processor, './../video.mp4')